In [1]:
from feature_engineering import *
msg = starts_with_letter('Аптека', 'vowel')
print(msg)

True


In [2]:
from data_loaders import load_processed_data

hogwarts_df = load_processed_data()

hogwarts_df.head()

,name,surname,is_english,name_starts_with_vowel,name_starts_with_consonant,name_ends_with_vowel,name_ends_with_consonant,name_length,name_vowels_count,name_double_vowels_count,...,surname_double_vowels_count,surname_consonant_count,surname_double_consonant_count,surname_paired_count,surname_deaf_count,surname_sonorus_count,is_griffindor,is_hufflpuff,is_ravenclaw,is_slitherin
0,Юан,Аберкромби,False,True,False,False,True,3,2,1,...,0,6,3,3,1,5,1,0,0,0
1,Кэти,Белл,False,False,True,True,False,4,2,0,...,0,3,1,1,0,3,1,0,0,0
2,Бем,,False,False,True,False,True,3,1,0,...,0,0,0,0,0,0,1,0,0,0
3,Сириус,Блэк,False,False,True,False,True,6,3,1,...,0,3,1,2,1,2,1,0,0,0
4,Лаванда,Браун,False,False,True,True,False,7,3,0,...,1,3,1,1,0,3,1,0,0,0


In [3]:
data_full = hogwarts_df.drop(columns=[
    'name', 
    'surname',
    'is_griffindor',
    'is_hufflpuff',
    'is_ravenclaw'
    ]).copy()
data_full.head()

,is_english,name_starts_with_vowel,name_starts_with_consonant,name_ends_with_vowel,name_ends_with_consonant,name_length,name_vowels_count,name_double_vowels_count,name_consonant_count,name_double_consonant_count,...,surname_ends_with_consonant,surname_length,surname_vowels_count,surname_double_vowels_count,surname_consonant_count,surname_double_consonant_count,surname_paired_count,surname_deaf_count,surname_sonorus_count,is_slitherin
0,False,True,False,False,True,3,2,1,1,0,...,False,10,4,0,6,3,3,1,5,0
1,False,False,True,True,False,4,2,0,2,0,...,True,4,1,0,3,1,1,0,3,0
2,False,False,True,False,True,3,1,0,2,0,...,False,0,0,0,0,0,0,0,0,0
3,False,False,True,False,True,6,3,1,3,0,...,True,4,1,0,3,1,2,1,2,0
4,False,False,True,True,False,7,3,0,4,1,...,True,5,2,1,3,1,1,0,3,0


In [4]:
X_data = data_full.drop(columns=['is_slitherin'])
# В качестве целевой будет колонка, которая содержит 1 для учеников Слизерина
y = data_full.is_slitherin

In [5]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from model_evaluation import score_testing_dataset

# Фиксируем сид для воспроизводимоси результата, random state
seed = 7
# Пропорции разделения датасета
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=test_size, random_state=seed)

rfc = RandomForestClassifier()
rfc_model = rfc.fit(X_train, y_train)

# взяли из model_evaluation файла метод оценку тестового набора имен, котрая там зашита, она вызывает на модели,
# котрую туда передает, rfc_model сделали predict_proba(encoded_person)[0]
score_testing_dataset(rfc_model)



Роман Арефьев — 0.2
Roman Arefev — 0.1
Светлана Кириченко — 0.5
Svetlana Kirichenko — 0.3
Гарри Поттер — 0.1
Harry Potter — 0.0
Северус Снейп — 1.0
Северус Снегг — 1.0
Severus Snape — 0.7
Том Реддл — 0.475
Tom Riddle — 0.6
Салазар Слизерин — 0.3
Salazar Slytherin — 0.6


/home/ra/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ra/.local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
predictions = rfc_model.predict(X_test)
print("Classification report: ")
print(classification_report(y_test, predictions))
print("Accuracy for Random Forest Model: %.2f" 
          % (accuracy_score(y_test, predictions) * 100))
print("ROC AUC from first Random Forest Model: %.2f"
             % (roc_auc_score(y_test, predictions)))

Classification report: 
             precision    recall  f1-score   support

          0       0.68      0.94      0.79       168
          1       0.62      0.18      0.28        89

avg / total       0.66      0.68      0.61       257

Accuracy for Random Forest Model: 67.70
ROC AUC from first Random Forest Model: 0.56


In [7]:
from model_training import train_classifiers
# from data_loaders import load_processed_data
import warnings
warnings.filterwarnings('ignore')

# Загружаем данные
hogwarts_df = load_processed_data()

# Оставляем только нужные колонки
data_full = hogwarts_df.drop(
    [
    'name', 
    'surname',
    'is_griffindor',
    'is_hufflpuff',
    'is_ravenclaw'
    ], 
    axis=1).copy()
X_data = data_full.drop('is_slitherin', axis=1)
y = data_full.is_slitherin

# Проводим исследование моделей
"""
Trains several classifiers and reporting model quality.
:param X_data:
:param y:
:return: trained models

return svm_model, svm_grid, \
           train_model1, xgb_grid, \
           rfc_model, rfc_grid, \
           ext_model, ext_grid, \
           lgbm_model, lgbm_grid, \
           rgf_model, rgf_grid, \
           frgf_model, frgf_grid

"""
slitherin_models = train_classifiers(X_data, y)
print(slitherin_models.__sizeof__())

# print(slitherin_models[5])
"""
    Scores models against pre-defined names
    :param models:
    :return: pd.DataFrame with probablities for each faculty

    сюда мы возвращаем rfc_grid предсказания для 5 срок котрые прописаны в методе в 
    модуле module_evaluation 
"""
score_testing_dataset(slitherin_models[1])

Random Forest Default Report
Accuracy for Random Forest Default: 70.82
ROC AUC for Random Forest Default: 0.54


Best params for Random Forest GridSearch: {'min_samples_leaf': 4, 'n_estimators': 100, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 10, 'max_depth': 5}
Score: 0.6500701381681707


Tuned Random Forest Report
Accuracy for Tuned Random Forest: 74.71
ROC AUC for Tuned Random Forest: 0.54


predict_proba for rfc_model
40
Роман Арефьев — 0.2559323121368683
Roman Arefev — 0.25619764662993333
Светлана Кириченко — 0.3805008244661133
Svetlana Kirichenko — 0.27383621929609786
Гарри Поттер — 0.2541784834242689
Harry Potter — 0.3461945278587659
Северус Снейп — 0.5732863412010643
Северус Снегг — 0.5960023413160814
Severus Snape — 0.4048382282086329
Том Реддл — 0.3360468207313256
Tom Riddle — 0.29089321779663363
Салазар Слизерин — 0.43984971437424364
Salazar Slytherin — 0.3480595884000208
